In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
plt.rcParams["font.family"]="Meiryo"

In [81]:
import warnings
# 不要な警告文非表示
warnings.filterwarnings("ignore")
from pycaret.regression import *

In [3]:
df=pd.read_csv('.\\data\\train\\01.csv')

In [4]:
df.head()

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,1004555,中古マンション等,NaN,1101,北海道,札幌市中央区,南２条西,西１８丁目,5,３ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2009年第2四半期,改装済,NaN,6.982271
1,1051274,中古マンション等,NaN,1106,北海道,札幌市南区,川沿１７条,真駒内,30分?60分,４ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2014年第3四半期,改装済,NaN,6.968483
2,1008483,中古マンション等,NaN,1101,北海道,札幌市中央区,南１４条西,西１８丁目,29,４ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2013年第3四半期,改装済,NaN,7.361728
3,1075401,中古マンション等,NaN,1110,北海道,札幌市清田区,清田２条,福住,30分?60分,４ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2013年第2四半期,未改装,NaN,6.991226
4,1090633,中古マンション等,NaN,1110,北海道,札幌市清田区,清田１条,福住,30分?60分,１ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,200.0,2008年第4四半期,未改装,NaN,6.579784


In [20]:
#全データを読み込んで結合
list_train = glob.glob('.\\data\\train\\*.csv')
df_concat=pd.DataFrame()

for fn in list_train:
    df=pd.read_csv(fn)
    df_concat=pd.concat([df_concat,df],axis=0)
df_concat.replace("2000㎡以上", "2000", inplace=True)
df_concat["面積（㎡）"]=df_concat["面積（㎡）"].astype(float)
df_concat["市区町村コード"]=df_concat["市区町村コード"].astype(str)
df_concat.reset_index(drop=True, inplace=True)


C:\Users\nutsf\AppData\Local\Temp\ipykernel_26520\388266807.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(fn)
C:\Users\nutsf\AppData\Local\Temp\ipykernel_26520\388266807.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(fn)
C:\Users\nutsf\AppData\Local\Temp\ipykernel_26520\388266807.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(fn)
C:\Users\nutsf\AppData\Local\Temp\ipykernel_26520\388266807.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(fn)


In [77]:
#建築年を西暦に変換
idx_reiwa=df_concat["建築年"][df_concat["建築年"].str.contains("令和",na=False)].index
idx_heisei=df_concat["建築年"][df_concat["建築年"].str.contains("平成",na=False)].index
idx_showa=df_concat["建築年"][df_concat["建築年"].str.contains("昭和",na=False)].index

df_concat.loc[idx_reiwa,"建築年"]=df_concat["建築年"][idx_reiwa].str.replace("令和","").str.replace("年","")
df_concat.loc[idx_reiwa,"建築年"]=df_concat["建築年"][idx_reiwa].astype(int)+2018

df_concat.loc[idx_heisei,"建築年"]=df_concat["建築年"][idx_heisei].str.replace("平成","").str.replace("年","")
df_concat.loc[idx_heisei,"建築年"]=df_concat["建築年"][idx_heisei].astype(int)+1988

df_concat.loc[idx_showa,"建築年"]=df_concat["建築年"][idx_showa].str.replace("昭和","").str.replace("年","")
df_concat.loc[idx_showa,"建築年"]=df_concat["建築年"][idx_showa].astype(int)+1925

#取引時点を西暦に変換
trade_period=df_concat["取引時点"].str[0:4].astype(int)
trade_period+=(df_concat["取引時点"].str[6:7].astype(int)-1)*0.25

#駅からの距離を数値に変換
df_concat.loc[:,"最寄駅：距離（分）"]=df_concat.loc[:,"最寄駅：距離（分）"].replace("30分?60分","45").replace("1H?1H30","75").replace("1H30?2H","105").replace("2H?","120").astype(float)

#改装を数値に変換
df_concat.loc[:,"改装"]=df_concat.loc[:,"改装"].replace("改装済","1").replace("未改装","0").fillna(0).astype(int)


C:\Users\nutsf\AppData\Local\Temp\ipykernel_26520\2043168938.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_concat.loc[:,"改装"]=df_concat.loc[:,"改装"].replace("改装済","1").replace("未改装","0").fillna(0).astype(int)


In [33]:
df_concat["今後の利用目的"].unique()

array([nan, '住宅', '事務所', 'その他', '店舗'], dtype=object)

In [34]:
print("事務所",(df_concat["今後の利用目的"]=="事務所").sum())
print("住宅",(df_concat["今後の利用目的"]=="住宅").sum())
print("店舗",(df_concat["今後の利用目的"]=="店舗").sum())
print("その他",(df_concat["今後の利用目的"]=="その他").sum())

#利用目的はほとんどが住宅用のため無視

事務所 4351
住宅 369313
店舗 1445
その他 22645


In [78]:
#教師データ作成

df_train=pd.DataFrame()
df_train["Year"]=df_concat["建築年"]
df_train["Area"]=df_concat["面積（㎡）"]
df_train["Distance"]=df_concat["最寄駅：距離（分）"]
df_train["Renovation"]=df_concat["改装"]
df_train["trade_period"]=trade_period
df_train["pref"]=df_concat["市区町村コード"].str[:-3].astype(int)
df_train["city"]=df_concat["市区町村コード"].str[-3:].astype(int)
df_train["price"]=df_concat["取引価格（総額）_log"]

In [79]:
df_train

,Year,Area,Distance,Renovation,trade_period,pref,city,price
0,1986,70.0,5.0,1,2009.25,1,101,6.982271
1,1995,90.0,45.0,1,2014.50,1,106,6.968483
2,1999,95.0,29.0,1,2013.50,1,101,7.361728
3,1996,80.0,45.0,0,2013.25,1,110,6.991226
4,1990,35.0,45.0,0,2008.75,1,110,6.579784
...,...,...,...,...,...,...,...,...
765407,1991,60.0,NaN,0,2010.25,47,201,7.000000
765408,2002,65.0,NaN,0,2015.25,47,211,7.230449
765409,2014,70.0,NaN,0,2017.25,47,208,7.462398
765410,2005,90.0,NaN,1,2018.00,47,208,7.397940


### Pycaret

In [83]:
reg1=setup(df_train, target = 'price', session_id=123)

,Description,Value
0,Session id,123
1,Target,price
2,Target type,Regression
3,Original data shape,"(765412, 8)"
4,Transformed data shape,"(765412, 8)"
5,Transformed train set shape,"(535788, 8)"
6,Transformed test set shape,"(229624, 8)"
7,Numeric features,6
8,Categorical features,1
9,Rows with missing values,5.8%


In [84]:
compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:21:08
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.1725,0.0581,0.2409,0.5447,0.0298,0.0243,3.1890
ridge,Ridge Regression,0.1725,0.0581,0.2409,0.5447,0.0298,0.0243,1.1920
lasso,Lasso Regression,0.2508,0.1046,0.3234,0.1798,0.0401,0.0353,1.9820


LGBMRegressor(random_state=123)

In [1]:
best=compare_models()

NameError: name 'compare_models' is not defined